In [33]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import random
import time
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import zipfile
import os
from pathlib import Path

#for graphing results
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

In [4]:
if not os.path.exists("data"):
    with zipfile.ZipFile("data.zip", "r") as zip_ref:
        zip_ref.extractall("data")

In [5]:
# hyperparameters
NUM_EPOCHS = 5
NUM_TRAINING = 1000
NUM_TESTING = 500
NUM_VALIDATION = 500

NUM_FACE_TRAINING = 451
NUM_FACE_VALIDATION = 301
NUM_FACE_TESTING = 150

IMAGE_HEIGHT = 28
IMAGE_WIDTH = 28
NUM_CLASSES = 10

In [6]:
# filepaths
train_data_file = "data/digitdata/trainingimages"
train_label_file = "data/digitdata/traininglabels"
val_data_file = "data/digitdata/validationimages"
val_label_file = "data/digitdata/validationlabels"
test_data_file = "data/digitdata/testimages"
test_label_file = "data/digitdata/testlabels"

face_train_data_file = "data/facedata/facedatatrain"
face_train_label_file = "data/facedata/facedatatrainlabels"
face_val_data_file   = "data/facedata/facedatavalidation"
face_val_label_file  = "data/facedata/facedatavalidationlabels"
face_test_data_file  = "data/facedata/facedatatest"
face_test_label_file = "data/facedata/facedatatestlabels"


## Data Loading and Preprocessing

In [7]:

def read_data_file(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    return [line.rstrip("\n") for line in lines]

def extract_features(raw_data):
    features = []
    for i in range(0, len(raw_data), 28):
        image = raw_data[i:i+28]
        feature = [1 if ch != ' ' else 0 for row in image for ch in row]
        features.append(feature)
    return features


def read_labels(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    return [int(line.strip()) for line in lines]

def load_dataset(data_file, label_file, size=None):
    raw_data = read_data_file(data_file)
    raw_labels = read_labels(label_file)

    features = extract_features(raw_data)
    if size is not None:
        combined = list(zip(features, raw_labels))
        random.shuffle(combined)
        features, raw_labels = zip(*combined[:size])

    return list(features), list(raw_labels)

def one_hot_encode(y, num_classes=10):
    encoded = np.zeros((num_classes, len(y)))
    for idx, val in enumerate(y):
        encoded[val][idx] = 1
    return encoded

def evaluate(predictions, labels):
    correct = sum(p == t for p, t in zip(predictions, labels))
    return correct / len(labels)


In [8]:
def extract_face_features(raw_data):
    features = []
    for i in range(0, len(raw_data), 70):  # 70 rows per image
        image = raw_data[i:i+70]
        assert all(len(row) == 60 for row in image), "Expected 60 columns per row in face image"
        feature = [1 if ch != ' ' else 0 for row in image for ch in row]
        features.append(feature)
    return features

def load_face_dataset(data_file, label_file, size=None):
    raw_data = read_data_file(data_file)
    raw_labels = read_labels(label_file)

    features = extract_face_features(raw_data)
    if size is not None:
        combined = list(zip(features, raw_labels))
        random.shuffle(combined)
        features, raw_labels = zip(*combined[:size])

    return list(features), list(raw_labels)

def one_hot_encode_face(y, num_classes=2):
    encoded = np.zeros((num_classes, len(y)))
    for idx, val in enumerate(y):
        encoded[val][idx] = 1
    return encoded


# Three-Layer Neural Network: Manual Implementations of Forward Pass, Back-Propagation, and Weight Update

## Neural Network Functions

In [34]:
import numpy as np

# Activation functions
def relu(z):
    return np.maximum(0, z)

def relu_derivative(z):
    return (z > 0).astype(float)

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(a):
    return a * (1 - a)

def softmax(Z):
    e_Z = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    return e_Z / np.sum(e_Z, axis=0, keepdims=True)


# Initialize weights and biases
def initialize_parameters(input_size, hidden1_size, hidden2_size, output_size):
    np.random.seed(42)
    return {
        'W1': np.random.randn(hidden1_size, input_size) * 0.01,
        'b1': np.zeros((hidden1_size, 1)),
        'W2': np.random.randn(hidden2_size, hidden1_size) * 0.01,
        'b2': np.zeros((hidden2_size, 1)),
        'W3': np.random.randn(output_size, hidden2_size) * 0.01,
        'b3': np.zeros((output_size, 1))
    }

# Forward pass
def forward_propagation(X, parameters):
    W1, b1 = parameters['W1'], parameters['b1']
    W2, b2 = parameters['W2'], parameters['b2']
    W3, b3 = parameters['W3'], parameters['b3']

    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)

    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)

    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)

    cache = (Z1, A1, Z2, A2, Z3, A3)
    return A3, cache


def forward_propagation_face(X, parameters, dropout_rate=0.5, training=True):
    W1, b1 = parameters['W1'], parameters['b1']
    W2, b2 = parameters['W2'], parameters['b2']
    W3, b3 = parameters['W3'], parameters['b3']

    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)

    if training:
        D1 = (np.random.rand(*A1.shape) < dropout_rate).astype(float)
        A1 *= D1
        A1 /= dropout_rate
    else:
        D1 = None

    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)

    if training:
        D2 = (np.random.rand(*A2.shape) < dropout_rate).astype(float)
        A2 *= D2
        A2 /= dropout_rate
    else:
        D2 = None

    Z3 = np.dot(W3, A2) + b3
    A3 = softmax(Z3)

    # Include dropout masks in the cache
    cache = (Z1, A1, D1, Z2, A2, D2, Z3, A3)
    return A3, cache



# Loss
def compute_loss(Y_hat, Y):
    m = Y.shape[1]
    return -np.sum(Y * np.log(Y_hat + 1e-8) + (1 - Y) * np.log(1 - Y_hat + 1e-8)) / m

def compute_loss_l2(Y_hat, Y, parameters, lambda_reg=0.1):
    m = Y.shape[1]
    cross_entropy = -np.sum(Y * np.log(Y_hat + 1e-8)) / m
    l2 = (lambda_reg / (2 * m)) * (
        np.sum(np.square(parameters['W1'])) +
        np.sum(np.square(parameters['W2'])) +
        np.sum(np.square(parameters['W3']))
    )
    return cross_entropy + l2

# Backward pass
def backward_propagation(X, Y, parameters, cache):
    m = X.shape[1]
    W2, W3 = parameters['W2'], parameters['W3']
    Z1, A1, Z2, A2, Z3, A3 = cache

    dZ3 = A3 - Y
    dW3 = (1/m) * np.dot(dZ3, A2.T)
    db3 = (1/m) * np.sum(dZ3, axis=1, keepdims=True)

    dA2 = np.dot(W3.T, dZ3)
    dZ2 = dA2 * relu_derivative(Z2)
    dW2 = (1/m) * np.dot(dZ2, A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)

    dA1 = np.dot(W2.T, dZ2)
    dZ1 = dA1 * relu_derivative(Z1)
    dW1 = (1/m) * np.dot(dZ1, X.T)
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)

    return {
        'dW1': dW1, 'db1': db1,
        'dW2': dW2, 'db2': db2,
        'dW3': dW3, 'db3': db3
    }


def backward_propagation_face(X, Y, parameters, cache, dropout_rate=0.5):
    m = X.shape[1]
    W2, W3 = parameters['W2'], parameters['W3']
    Z1, A1, D1, Z2, A2, D2, Z3, A3 = cache

    dZ3 = A3 - Y
    dW3 = (1/m) * np.dot(dZ3, A2.T)
    db3 = (1/m) * np.sum(dZ3, axis=1, keepdims=True)

    dA2 = np.dot(W3.T, dZ3)
    dA2 *= D2
    dA2 /= dropout_rate
    dZ2 = dA2 * relu_derivative(Z2)
    dW2 = (1/m) * np.dot(dZ2, A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)

    dA1 = np.dot(W2.T, dZ2)
    dA1 *= D1
    dA1 /= dropout_rate
    dZ1 = dA1 * relu_derivative(Z1)
    dW1 = (1/m) * np.dot(dZ1, X.T)
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)

    return {
        'dW1': dW1, 'db1': db1,
        'dW2': dW2, 'db2': db2,
        'dW3': dW3, 'db3': db3
    }

# Gradient descent update
def update_parameters(params, grads, lr):
    for key in params:
        params[key] -= lr * grads['d' + key]
    return params

# Prediction
def predict_nn(X, parameters):
    Y_hat, _ = forward_propagation(X, parameters)
    return np.argmax(Y_hat, axis=0)

def predict_nn_face(X, parameters):
    Y_hat, _ = forward_propagation_face(X, parameters, training=False)
    return np.argmax(Y_hat, axis=0)

# Training loop
def train_neural_net(X_train, y_train, X_test, y_test,
                     input_size, h1, h2, output_size,
                     epochs=1000, lr=0.1, print_loss=True,
                     X_val=None, y_val=None, early_stopping=False, patience=10):
    
    start_time = time.time()
    
    parameters = initialize_parameters(input_size, h1, h2, output_size)
    best_params = None
    best_val_acc = 0
    val_acc_counter = 0

    for epoch in range(epochs):
        # Forward and backpropagation
        Y_hat, cache = forward_propagation(X_train, parameters)
        loss = compute_loss(Y_hat, y_train)
        grads = backward_propagation(X_train, y_train, parameters, cache)
        parameters = update_parameters(parameters, grads, lr)

        # Check performance every 100 epochs
        if epoch % 100 == 0 or epoch == epochs - 1:
            train_preds = predict_nn(X_train, parameters)
            train_acc = evaluate(train_preds, np.argmax(y_train, axis=0))
            
            if X_val is not None and y_val is not None:
                val_preds = predict_nn(X_val, parameters)
                val_acc = evaluate(val_preds, np.argmax(y_val, axis=0))

                if print_loss:
                    print(f"Epoch {epoch}: Loss = {loss:.4f} | Train Acc = {train_acc:.4f} | Val Acc = {val_acc:.4f}")

                # Save best model
                if val_acc > best_val_acc:
                    best_val_acc = val_acc
                    best_params = {k: v.copy() for k, v in parameters.items()}
                    val_acc_counter = 0
                else:
                    val_acc_counter += 1
                    if early_stopping and val_acc_counter >= patience:
                        print("Early stopping triggered.")
                        break
            else:
                if print_loss:
                    print(f"Epoch {epoch}: Loss = {loss:.4f} | Train Acc = {train_acc:.4f}")
                    
    end_time = time.time()
    training_time = end_time - start_time  

    final_params = best_params if best_params is not None else parameters

    # Final test evaluation
    test_preds = predict_nn(X_test, final_params)
    test_acc = evaluate(test_preds, np.argmax(y_test, axis=0))
    print(f"Final Test Accuracy: {test_acc:.4f}")
    print(f"Training Time: {training_time:.2f} seconds")
    
    return final_params, training_time



def train_neural_net_face(X_train, y_train, X_test, y_test,
                     input_size, h1, h2, output_size,
                     epochs=1000, lr=0.1, print_loss=True,
                     X_val=None, y_val=None, early_stopping=False,
                     patience=10, dropout_rate=0.5, lambda_reg=0.1):  
    start_time = time.time()
    
    parameters = initialize_parameters(input_size, h1, h2, output_size)
    best_params = None
    best_val_acc = 0
    val_acc_counter = 0

    for epoch in range(epochs):
        # === DROPOUT + L2 ===
        Y_hat, cache = forward_propagation_face(X_train, parameters, dropout_rate=dropout_rate, training=True)
        loss = compute_loss_l2(Y_hat, y_train, parameters, lambda_reg=lambda_reg)
        grads = backward_propagation_face(X_train, y_train, parameters, cache, dropout_rate=dropout_rate)
        parameters = update_parameters(parameters, grads, lr)

        if epoch % 100 == 0 or epoch == epochs - 1:
            train_preds = predict_nn_face(X_train, parameters)
            train_acc = evaluate(train_preds, np.argmax(y_train, axis=0))

            if X_val is not None and y_val is not None:
                val_preds = predict_nn_face(X_val, parameters)
                val_acc = evaluate(val_preds, np.argmax(y_val, axis=0))

                if print_loss:
                    print(f"Epoch {epoch}: Loss = {loss:.4f} | Train Acc = {train_acc:.4f} | Val Acc = {val_acc:.4f}")

                if val_acc > best_val_acc:
                    best_val_acc = val_acc
                    best_params = {k: v.copy() for k, v in parameters.items()}
                    val_acc_counter = 0
                else:
                    val_acc_counter += 1
                    if early_stopping and val_acc_counter >= patience:
                        print("Early stopping triggered.")
                        break
            else:
                if print_loss:
                    print(f"Epoch {epoch}: Loss = {loss:.4f} | Train Acc = {train_acc:.4f}")
                    
    end_time = time.time()  
    training_time = end_time - start_time  

    final_params = best_params if best_params is not None else parameters
    test_preds = predict_nn_face(X_test, final_params)
    test_acc = evaluate(test_preds, np.argmax(y_test, axis=0))
    print(f"Final Test Accuracy: {test_acc:.4f}")
    print(f"Training Time: {training_time:.2f} seconds")  
    
    return final_params, training_time


## Digit Classification

In [35]:
print("\n=== Testing neural net on DIGIT data ===")

# Load and preprocess digit data
X_train_raw, y_train_raw = load_dataset(train_data_file, train_label_file, size=NUM_TRAINING)
X_val_raw,   y_val_raw   = load_dataset(val_data_file,   val_label_file,   size=NUM_VALIDATION)
X_test_raw,  y_test_raw  = load_dataset(test_data_file,  test_label_file,  size=NUM_TESTING)

X_train = np.array(X_train_raw).T
X_val   = np.array(X_val_raw).T
X_test  = np.array(X_test_raw).T

y_train = one_hot_encode(y_train_raw)
y_val   = one_hot_encode(y_val_raw)
y_test  = one_hot_encode(y_test_raw)

# Experiment parameters
pct_list   = [0.1 * i for i in range(1, 11)]      # 10 % … 100 %
num_trials = 5
N          = X_train.shape[1]
indices    = np.arange(N)

raw_rows = []                                      # one row per trial


# Main loop: each percentage ➜ five random trials

for pct in pct_list:
    k = int(pct * N)

    for trial in range(num_trials):
        np.random.shuffle(indices)                 # fresh subset
        idx      = indices[:k]
        X_sub    = X_train[:, idx]
        y_sub    = y_train[:, idx]

        # train on the subset 
        t0 = time.time()
        params, _ = train_neural_net(
            X_sub, y_sub,
            X_test, y_test,
            input_size=784, h1=128, h2=64, output_size=10,
            epochs=1000, lr=0.1,
            X_val=X_val, y_val=y_val,
            early_stopping=True, patience=10
        )
        train_sec = time.time() - t0

        # evaluate on held‑out test & validation sets 
        test_preds = predict_nn(X_test, params)
        test_err   = 1.0 - evaluate(test_preds, np.argmax(y_test, axis=0))

        val_preds  = predict_nn(X_val, params)
        val_acc    = evaluate(val_preds, np.argmax(y_val, axis=0))

        # store raw trial 
        raw_rows.append({
            "pct":         pct,          # 0.1 … 1.0
            "samples":     k,
            "trial":       trial + 1,
            "train_sec":   train_sec,
            "test_err":    test_err,
            "val_acc":     val_acc
        })

        print(f"{int(pct*100):3d}% | trial {trial+1} | "
              f"err={test_err:.4f} | val={val_acc:.4f} | time={train_sec:.2f}s")

# Aggregate: mean ± std for each percentage
df_raw = pd.DataFrame(raw_rows)

df_summary = (df_raw
              .groupby("pct")
              .agg(mean_train_sec=("train_sec", "mean"),
                   mean_err      =("test_err",  "mean"),
                   std_err       =("test_err",  "std"),
                   mean_val_acc  =("val_acc",   "mean"),
                   std_val_acc   =("val_acc",   "std"))
              .reset_index())

print("\nDigit NN summary (mean over 5 trials)")
print(df_summary.round(4).to_string(index=False))

# Save CSVs for later plotting / report
df_raw.to_csv("digit_nn_all_trials.csv", index=False)
df_summary.to_csv("digit_nn_summary.csv", index=False)



DIGITS: Training on 100 samples (10%)
Epoch 0: Loss = 6.9313 | Train Acc = 0.1700 | Val Acc = 0.0960
Epoch 100: Loss = 3.1770 | Train Acc = 0.1600 | Val Acc = 0.0960
Epoch 200: Loss = 3.0057 | Train Acc = 0.2200 | Val Acc = 0.1360
Epoch 300: Loss = 1.3479 | Train Acc = 0.7100 | Val Acc = 0.4440
Epoch 400: Loss = 0.3037 | Train Acc = 1.0000 | Val Acc = 0.5320
Epoch 500: Loss = 0.0585 | Train Acc = 1.0000 | Val Acc = 0.5580
Epoch 600: Loss = 0.0243 | Train Acc = 1.0000 | Val Acc = 0.5620
Epoch 700: Loss = 0.0141 | Train Acc = 1.0000 | Val Acc = 0.5680
Epoch 800: Loss = 0.0095 | Train Acc = 1.0000 | Val Acc = 0.5700
Epoch 900: Loss = 0.0071 | Train Acc = 1.0000 | Val Acc = 0.5720
Epoch 999: Loss = 0.0055 | Train Acc = 1.0000 | Val Acc = 0.5760
Final Test Accuracy: 0.5620
Training Time: 1.96 seconds
DIGITS Test Accuracy with 100 samples: 0.5620
Training Time: 1.96 seconds

Digit Classification Results:
Model  Samples  Data %  Training Time  Test Accuracy  Validation Accuracy  Epoch
Digit 

## Face Classification

In [ ]:
print("\n=== Testing neural net on FACE data ===")

# Load and preprocess face data
X_face_train_raw, y_face_train_raw = load_face_dataset(
    face_train_data_file, face_train_label_file, size=NUM_FACE_TRAINING)
X_face_val_raw,   y_face_val_raw   = load_face_dataset(
    face_val_data_file,  face_val_label_file,  size=NUM_FACE_VALIDATION)
X_face_test_raw,  y_face_test_raw  = load_face_dataset(
    face_test_data_file, face_test_label_file, size=NUM_FACE_TESTING)

X_face_train = np.array(X_face_train_raw).T
X_face_val   = np.array(X_face_val_raw).T
X_face_test  = np.array(X_face_test_raw).T

y_face_train = one_hot_encode_face(y_face_train_raw)
y_face_val   = one_hot_encode_face(y_face_val_raw)
y_face_test  = one_hot_encode_face(y_face_test_raw)

# Experiment parameters
pct_list   = [0.1 * i for i in range(1, 11)]      # 10 % … 100 %
num_trials = 5
N_faces    = X_face_train.shape[1]
indices    = np.arange(N_faces)

raw_rows_face = []                                # one row per trial

# Main loop: each percentage ➜ five random trials
for pct in pct_list:
    k = int(pct * N_faces)

    for trial in range(num_trials):
        np.random.shuffle(indices)                # fresh subset
        idx   = indices[:k]
        X_sub = X_face_train[:, idx]
        y_sub = y_face_train[:, idx]

        # train on the subset 
        t0 = time.time()
        params, _ = train_neural_net_face(
            X_sub, y_sub,
            X_face_test, y_face_test,
            input_size=4200, h1=32, h2=16, output_size=2,
            epochs=1000, lr=0.1,
            X_val=X_face_val, y_val=y_face_val,
            early_stopping=True, patience=10,
            dropout_rate=0.5, lambda_reg=0.5
        )
        train_sec = time.time() - t0

        # evaluate on test & validation sets 
        test_preds = predict_nn_face(X_face_test, params)
        test_err   = 1.0 - evaluate(test_preds, np.argmax(y_face_test, axis=0))

        val_preds  = predict_nn_face(X_face_val, params)
        val_acc    = evaluate(val_preds, np.argmax(y_face_val, axis=0))

        # store raw trial 
        raw_rows_face.append({
            "pct":         pct,        # 0.1 … 1.0
            "samples":     k,
            "trial":       trial + 1,
            "train_sec":   train_sec,
            "test_err":    test_err,
            "val_acc":     val_acc
        })

        print(f"{int(pct*100):3d}% | trial {trial+1} | "
              f"err={test_err:.4f} | val={val_acc:.4f} | time={train_sec:.2f}s")

# Aggregate: mean ± std for each percentage
df_face_raw = pd.DataFrame(raw_rows_face)

df_face_summary = (df_face_raw
                   .groupby("pct")
                   .agg(mean_train_sec=("train_sec", "mean"),
                        mean_err      =("test_err",  "mean"),
                        std_err       =("test_err",  "std"),
                        mean_val_acc  =("val_acc",   "mean"),
                        std_val_acc   =("val_acc",   "std"))
                   .reset_index())

print("\nFace NN summary (mean over 5 trials)")
print(df_face_summary.round(4).to_string(index=False))


# Save CSVs for later plotting / report
df_face_raw.to_csv("face_nn_all_trials.csv", index=False)
df_face_summary.to_csv("face_nn_summary.csv", index=False)



=== Testing neural net on FACE data ===
Epoch 0: Loss = 0.7682 | Train Acc = 0.5556 | Val Acc = 0.4817
Epoch 100: Loss = 0.7611 | Train Acc = 0.5556 | Val Acc = 0.4817
Epoch 200: Loss = 0.2826 | Train Acc = 1.0000 | Val Acc = 0.6246
Epoch 300: Loss = 0.1417 | Train Acc = 1.0000 | Val Acc = 0.6678
Epoch 400: Loss = 0.1211 | Train Acc = 1.0000 | Val Acc = 0.6711
Epoch 500: Loss = 0.1301 | Train Acc = 1.0000 | Val Acc = 0.6910
Epoch 600: Loss = 0.1283 | Train Acc = 1.0000 | Val Acc = 0.6844
Epoch 700: Loss = 0.1406 | Train Acc = 1.0000 | Val Acc = 0.6977
Epoch 800: Loss = 0.1395 | Train Acc = 1.0000 | Val Acc = 0.7010
Epoch 900: Loss = 0.1366 | Train Acc = 1.0000 | Val Acc = 0.6811
Epoch 999: Loss = 0.1445 | Train Acc = 1.0000 | Val Acc = 0.7010
Final Test Accuracy: 0.7467
Training Time: 0.84 seconds
 10% | trial 1 | err=0.2533 | val=0.7010 | time=0.84s
Epoch 0: Loss = 0.7681 | Train Acc = 0.5111 | Val Acc = 0.5183
Epoch 100: Loss = 0.7675 | Train Acc = 0.5111 | Val Acc = 0.5183
Epoch 20

In [ ]:
# Use clean Seaborn style
sns.set(style="whitegrid", context="talk")

# Load trial-level raw data
def load_trial_data():
    df_digit = pd.read_csv("digit_nn_all_trials.csv")
    df_face  = pd.read_csv("face_nn_all_trials.csv")

    df_digit["Model"] = "Digit"
    df_face["Model"]  = "Face"

    df_all = pd.concat([df_digit, df_face], ignore_index=True)

    # Rename columns for consistency
    df_all = df_all.rename(columns={
        "pct": "Data %",
        "test_err": "Test Error",
        "val_acc": "Validation Accuracy",
        "train_sec": "Training Time"
    })

    # Convert error to accuracy
    df_all["Test Accuracy"] = 1.0 - df_all["Test Error"]

    return df_all

# Analyze and plot using Seaborn (shaded std dev bands)
def analyze_performance(df, output_dir="Charts-and-Graphs/Part_B"):
    os.makedirs(output_dir, exist_ok=True)
    for f in os.listdir(output_dir):
        if f.startswith("partB_") and f.endswith(".pdf"):
            os.remove(os.path.join(output_dir, f))

    # learning curves 
    with PdfPages(os.path.join(output_dir, "partB_learning_curves.pdf")) as pdf:
        plt.figure(figsize=(10, 6))
        sns.lineplot(
            data=df,
            x="Data %", y="Test Accuracy", hue="Model",
            style="Model", markers=True, dashes=False,
            errorbar="sd", err_style="band"
        )
        plt.title("Learning Curves with Standard Deviation")
        plt.xlabel("% Training Data")
        plt.ylabel("Test Accuracy")
        plt.grid(True, linestyle="--", alpha=0.7)
        plt.legend()
        pdf.savefig(bbox_inches="tight")
        plt.close()

    # training time curves 
    with PdfPages(os.path.join(output_dir, "partB_training_times.pdf")) as pdf:
        plt.figure(figsize=(10, 6))
        sns.lineplot(
            data=df,
            x="Data %", y="Training Time", hue="Model",
            style="Model", markers=True, dashes=False,
            errorbar="sd", err_style="band"
        )
        plt.title("Training Time vs Data Size (with Std Dev)")
        plt.xlabel("% Training Data")
        plt.ylabel("Training Time (s)")
        plt.grid(True, linestyle="--", alpha=0.7)
        plt.legend()
        pdf.savefig(bbox_inches="tight")
        plt.close()

    # console summary 
    print("\nNeural Network Part B – Performance Summary")
    for model in ["Digit", "Face"]:
        mdf = df[df["Model"] == model]
        print(f"\n{model} Recognition:")
        print(f"Average Training Time: {mdf['Training Time'].mean():.2f} ± {mdf['Training Time'].std():.2f} s")
        print(f"Average Test Accuracy: {mdf['Test Accuracy'].mean():.4f} ± {mdf['Test Accuracy'].std():.4f}")

# Run the analysis
summary_df = load_trial_data()
analyze_performance(summary_df)



Neural Network Part B – Performance Summary

Digit Recognition:
Average Training Time: 3.35 ± 1.57 s
Average Test Accuracy: 0.7201 ± 0.0524

Face Recognition:
Average Training Time: 4.28 ± 2.07 s
Average Test Accuracy: 0.8336 ± 0.0622


# Three-Layer Neural Network: PyTorch Implementation

## Helper Functions

In [9]:
def to_tensor(x, dtype=torch.float32):
    return torch.tensor(np.asarray(x), dtype=dtype)

def make_loader(X, y, batch_size=32, shuffle=True):
    ds = TensorDataset(to_tensor(X), torch.tensor(y, dtype=torch.long))
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle)

def accuracy(logits, y):
    return (logits.argmax(1) == y).float().mean().item()

## Neural Network Functions

In [10]:
class ClassificationNetwork(nn.Module):
    def __init__(self, in_shape, n_out, dropout=0.5): # in_shape formatted like (channels, height, width)
        super().__init__()
        C, H, W = in_shape
        self.features = nn.Sequential(
            nn.Unflatten(dim=1, unflattened_size=in_shape),
            nn.Conv2d(C, 32, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )

        H2, W2 = H // 2, W // 2 # after pool, divide by 2
        fc_in  = 32 * H2 * W2

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(p=dropout),
            nn.Linear(fc_in, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, n_out) # output layer
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

In [14]:
def get_accuracy(model, loader, device="cpu"):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb).argmax(1)
            correct += (pred == yb).sum().item()
            total += yb.size(0)
    return correct / total

# training loop
def train(model, train_loader, val_loader, epochs=200, lr=1e-3, patience=15, weight_decay=1e-4, log_every=10, device="cpu"):
    model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    loss_func = nn.CrossEntropyLoss()

    best_val_acc, best_state, bad_epochs = 0.0, None, 0
    start_time = time.perf_counter()
    for epoch in range(1, epochs + 1):
        model.train()
        running_loss = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad()
            loss = loss_func(model(xb), yb)
            loss.backward()
            opt.step()
            running_loss += loss.item() * yb.size(0)

        train_acc = get_accuracy(model, train_loader, device)
        val_acc = get_accuracy(model, val_loader, device)

        if epoch % log_every == 0 or epoch == epochs:
            avg_loss = running_loss / len(train_loader.dataset)
            print(f"Epoch {epoch:3d}: "
                  f"Loss = {avg_loss:.4f} | "
                  f"Train Acc = {train_acc:.4f} | "
                  f"Val Acc = {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc, best_state = val_acc, model.state_dict()
            bad_epochs = 0
        else:
            bad_epochs += 1
            if bad_epochs >= patience:
                print("Early stopping triggered.")
                break

    total_time = time.perf_counter() - start_time
    model.load_state_dict(best_state)
    return model, total_time, best_val_acc

def test_neural_net(X_train, y_train, X_val, y_val, X_test, y_test, channels, height, width, n_out,
                    dropout, lr, epochs, weight_decay, patience):
    results = []
    fractions = np.linspace(0.1, 1.0, 10)
    for frac in fractions:
        n = int(frac * len(y_train))
        print(f"\nTraining on {n} samples ({frac:.0%})")
        train_loader = make_loader(X_train[:n], y_train[:n])
        val_loader = make_loader(X_val, y_val, shuffle=True)
        test_loader = make_loader(X_test, y_test, shuffle=True)

        model = ClassificationNetwork((channels, height, width), n_out)
        model, total_time, _ = train(model, train_loader, val_loader, lr=lr, epochs=epochs, weight_decay=weight_decay, patience=patience)

        # final test
        model.eval()
        with torch.no_grad():
            test_acc = np.mean([
                accuracy(model(xb), yb) for xb, yb in test_loader
            ])

        print(f"Final Test Accuracy: {test_acc:.4f}")
        results.append((n, frac, total_time, test_acc))
    return results

### Performance Analysis Functions

In [27]:
def process_results(results):
    all_results = []
    for run_idx, run_results in enumerate(results):
        for n, frac, total_time, test_acc in run_results:
            all_results.append({
                'run': run_idx,
                'n_samples': n,
                'fraction': frac,
                'training_time': total_time,
                'test_accuracy': test_acc,
                'prediction_error': 1 - test_acc
            })
    
    df = pd.DataFrame(all_results)
    stats_df = df.groupby('fraction').agg({
        'training_time': ['mean', 'std'],
        'test_accuracy': ['mean', 'std'],
        'prediction_error': ['mean', 'std']
    }).reset_index()
    
    stats_df.columns = ['_'.join(col).strip('_') for col in stats_df.columns.values]
    
    return df, stats_df

def create_visualizations(df, stats_df, classification_type):
    output_dir = Path('Charts-and-Graphs')
    output_dir.mkdir(exist_ok=True)
    plt.style.use('seaborn')
    
    # 1. Learning Curves
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df, x='fraction', y='test_accuracy', 
                errorbar='sd', label='Test Accuracy')
    plt.title('Learning Curves - Test Accuracy vs Fraction of Samples')
    plt.xlabel('Fraction of Training Samples')
    plt.ylabel('Test Accuracy')
    plt.grid(True)
    plt.savefig(output_dir / f'partC_{classification_type}_learning_curves.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # 2. Training Time
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df, x='fraction', y='training_time',
                errorbar='sd', label='Training Time')
    plt.title('Training Time vs Fraction of Samples')
    plt.xlabel('Fraction of Training Samples')
    plt.ylabel('Training Time (seconds)')
    plt.grid(True)
    plt.savefig(output_dir / f'partC_{classification_type}_training_times.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    stats_df.to_csv(output_dir / f'partC_{classification_type}_neural_network_results.csv', index=False)
    display_df = stats_df.copy()
    display_df.columns = ['Fraction', 
                         'Mean Training Time (s)', 'Std Training Time (s)',
                         'Mean Test Accuracy', 'Std Test Accuracy',
                         'Mean Prediction Error', 'Std Prediction Error']
    display_df.to_csv(output_dir / f'partC_{classification_type}_formatted_results.csv', index=False)

def create_combined_visualizations(digits_df, face_df, digits_stats_df, face_stats_df):
    output_dir = Path('Charts-and-Graphs')
    output_dir.mkdir(exist_ok=True)
    plt.style.use('seaborn')
    
    plt.figure(figsize=(12, 7))
    sns.lineplot(data=digits_df, x='fraction', y='test_accuracy', 
                errorbar='sd', label='Digit Classification')
    sns.lineplot(data=face_df, x='fraction', y='test_accuracy',
                errorbar='sd', label='Face Classification')
    plt.title('Learning Curves - Test Accuracy vs Fraction of Samples')
    plt.xlabel('Fraction of Training Samples')
    plt.ylabel('Test Accuracy')
    plt.grid(True)
    plt.legend()
    plt.savefig(output_dir / 'partC_combined_learning_curves.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    plt.figure(figsize=(12, 7))
    sns.lineplot(data=digits_df, x='fraction', y='training_time',
                errorbar='sd', label='Digit Classification')
    sns.lineplot(data=face_df, x='fraction', y='training_time',
                errorbar='sd', label='Face Classification')
    plt.title('Training Time vs Fraction of Samples')
    plt.xlabel('Fraction of Training Samples')
    plt.ylabel('Training Time (seconds)')
    plt.grid(True)
    plt.legend()
    plt.savefig(output_dir / 'partC_combined_training_times.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    combined_stats = pd.concat([
        digits_stats_df.assign(Model='Digit Classification'),
        face_stats_df.assign(Model='Face Classification')
    ])
    
    combined_stats.to_csv(output_dir / 'partC_combined_neural_network_results.csv', index=False)
    
    display_df = combined_stats.copy()
    display_df.columns = ['Fraction', 
                         'Mean Training Time (s)', 'Std Training Time (s)',
                         'Mean Test Accuracy', 'Std Test Accuracy',
                         'Mean Prediction Error', 'Std Prediction Error',
                         'Model']
    display_df.to_csv(output_dir / 'partC_combined_formatted_results.csv', index=False)

## Digit Classification

In [22]:
X_digit_train_raw, y_digit_train_raw = load_dataset(train_data_file, train_label_file, size=NUM_TRAINING)
X_digit_val_raw, y_digit_val_raw = load_dataset(val_data_file, val_label_file, size=NUM_VALIDATION)
X_digit_test_raw, y_digit_test_raw = load_dataset(test_data_file, test_label_file, size=NUM_TESTING)

Xd_train, yd_train = np.array(X_digit_train_raw), np.array(y_digit_train_raw)
Xd_val, yd_val = np.array(X_digit_val_raw), np.array(y_digit_val_raw)
Xd_test, yd_test = np.array(X_digit_test_raw), np.array(y_digit_test_raw)

digit_results = []
for i in range(5):
    digit_results.append(test_neural_net(
        Xd_train, yd_train, Xd_val, yd_val, Xd_test, yd_test,
        channels=1, height=28, width=28, n_out=10,
        dropout=0.1,
        lr=0.005, epochs=100,
        weight_decay=1e-4, patience=15
    ))


Training on 100 samples (10%)
Epoch  10: Loss = 0.4784 | Train Acc = 0.8900 | Val Acc = 0.6320
Epoch  20: Loss = 0.0251 | Train Acc = 1.0000 | Val Acc = 0.7240
Epoch  30: Loss = 0.0074 | Train Acc = 1.0000 | Val Acc = 0.7560
Epoch  40: Loss = 0.0022 | Train Acc = 1.0000 | Val Acc = 0.7400
Early stopping triggered.
Final Test Accuracy: 0.6992

Training on 200 samples (20%)
Epoch  10: Loss = 0.0456 | Train Acc = 1.0000 | Val Acc = 0.8140
Epoch  20: Loss = 0.0051 | Train Acc = 1.0000 | Val Acc = 0.8060
Early stopping triggered.
Final Test Accuracy: 0.7793

Training on 300 samples (30%)
Epoch  10: Loss = 0.0681 | Train Acc = 0.9833 | Val Acc = 0.8060
Epoch  20: Loss = 0.0051 | Train Acc = 1.0000 | Val Acc = 0.8700
Epoch  30: Loss = 0.0108 | Train Acc = 1.0000 | Val Acc = 0.8600
Early stopping triggered.
Final Test Accuracy: 0.8293

Training on 400 samples (40%)
Epoch  10: Loss = 0.0178 | Train Acc = 1.0000 | Val Acc = 0.8600
Epoch  20: Loss = 0.0174 | Train Acc = 1.0000 | Val Acc = 0.8540

In [28]:
digits_df, digits_stats_df = process_results(digit_results)
create_visualizations(digits_df, digits_stats_df, 'digits')

/var/folders/s1/dzlvnvfs33bgbvhb6d13h9rh0000gn/T/ipykernel_59853/1836228052.py:28: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


## Face Classification

In [24]:
X_face_train_raw, y_face_train_raw = load_face_dataset(face_train_data_file, face_train_label_file, size=NUM_FACE_TRAINING)
X_face_val_raw, y_face_val_raw = load_face_dataset(face_val_data_file, face_val_label_file, size=NUM_FACE_VALIDATION)
X_face_test_raw, y_face_test_raw = load_face_dataset(face_test_data_file, face_test_label_file, size=NUM_FACE_TESTING)

Xd_train, yd_train = np.array(X_face_train_raw), np.array(y_face_train_raw)
Xd_val, yd_val = np.array(X_face_val_raw), np.array(y_face_val_raw)
Xd_test, yd_test = np.array(X_face_test_raw), np.array(y_face_test_raw)

face_results = []
for i in range(5):
    face_results.append(test_neural_net(
        Xd_train, yd_train, Xd_val, yd_val, Xd_test, yd_test,
        channels=1, height=70, width=60, n_out=2,
        dropout=0.3,
        lr=0.01, epochs=100,
        weight_decay=1e-3, patience=15
    ))


Training on 45 samples (10%)
Epoch  10: Loss = 0.0298 | Train Acc = 1.0000 | Val Acc = 0.8738
Epoch  20: Loss = 0.0016 | Train Acc = 1.0000 | Val Acc = 0.8837
Epoch  30: Loss = 0.0013 | Train Acc = 1.0000 | Val Acc = 0.9369
Epoch  40: Loss = 0.0002 | Train Acc = 1.0000 | Val Acc = 0.7774
Early stopping triggered.
Final Test Accuracy: 0.8977

Training on 90 samples (20%)
Epoch  10: Loss = 0.1804 | Train Acc = 1.0000 | Val Acc = 0.7342
Epoch  20: Loss = 0.0022 | Train Acc = 1.0000 | Val Acc = 0.9269
Epoch  30: Loss = 0.0012 | Train Acc = 1.0000 | Val Acc = 0.9502
Early stopping triggered.
Final Test Accuracy: 0.9290

Training on 135 samples (30%)
Epoch  10: Loss = 0.0078 | Train Acc = 1.0000 | Val Acc = 0.9302
Epoch  20: Loss = 0.0049 | Train Acc = 1.0000 | Val Acc = 0.9601
Epoch  30: Loss = 0.0041 | Train Acc = 1.0000 | Val Acc = 0.9535
Early stopping triggered.
Final Test Accuracy: 0.9040

Training on 180 samples (40%)
Epoch  10: Loss = 0.2553 | Train Acc = 1.0000 | Val Acc = 0.9169
E

In [29]:
face_df, face_stats_df = process_results(face_results)
create_visualizations(face_df, face_stats_df, 'face')

/var/folders/s1/dzlvnvfs33bgbvhb6d13h9rh0000gn/T/ipykernel_59853/1836228052.py:28: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [30]:
# combined results
create_combined_visualizations(digits_df, face_df, digits_stats_df, face_stats_df)

/var/folders/s1/dzlvnvfs33bgbvhb6d13h9rh0000gn/T/ipykernel_59853/1836228052.py:63: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')
